In [10]:
import torch

a = torch.LongTensor([[1,2,3,4,5,6,0,0,0], [1,2,3,4,5,6,7,0,0]])
b = torch.LongTensor([[1,2,3,4,5,6,7,8,9,10,11,12,0,0,0], [1,2,3,4,5,6,7,8,9,0,0,0,0,0,0]])

c = (a != 0).long()
print(c)

d = (b != 0).bool()
print(d)
e = torch.gather(a, 0, index=c)
print(e)

tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0]])
tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True, False,
         False, False, False, False, False]])
tensor([[1, 2, 3, 4, 5, 6, 0, 0, 0],
        [1, 2, 3, 4, 5, 6, 7, 0, 0]])


In [3]:
import torch
from transformers import BartTokenizer
MAX_SENT_LEN = 10

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base', max_len=MAX_SENT_LEN)

ids = tokenizer.encode('I am a student. I am a student. I am a student. I am a student.', add_special_tokens=False)
print(ids)
ids = tokenizer.encode('I am a student. I am a student. I am a student. I am a student.', add_special_tokens=False, max_length=MAX_SENT_LEN)
print(ids)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[649, 21402, 100, 4236, 21402, 424, 4236, 21402, 102, 4236, 21402, 34935, 4]
[100, 524, 10, 1294, 4]


In [140]:
import random
def _entity_aware_text_masking(text, entity):
    entity_tokens = entity.split(' ')
    text_tokens = text.split(' ')
    replace_text_tokens = []

    # random mask some entities
    i = 0
    while i < len(text_tokens):
        if ' '.join(entity_tokens).lower() == ' '.join(text_tokens[i:i+(len(entity_tokens))]).lower():
            if random.random() < 0.8:
                replace_text_tokens.append('<mask>')
                i += len(entity_tokens)
            else:
                replace_text_tokens.append(text_tokens[i])
                i += 1
        else:
            replace_text_tokens.append(text_tokens[i])
            i += 1

    return ' '.join(replace_text_tokens)

def _token_aware_text_masking(text):
    replace_text_tokens = text.split(' ')
    # random mask some tokens
    for i, _token in enumerate(replace_text_tokens):
        if random.random() < 0.1:
            replace_text_tokens[i] = '<mask>'

    return ' '.join(replace_text_tokens)

In [152]:
text = 'Beijing is a beautiful city. And Machial Jordan wife went to there twice.'
entities = ['Machial Jordan', 'Beijing']
for entity in entities:
    text = _entity_aware_text_masking(text, entity)
    print(text)
text = _token_aware_text_masking(text)
print(text)

Beijing is a beautiful city. And <mask> wife went to there twice.
<mask> is a beautiful city. And <mask> wife went to there twice.
<mask> is <mask> beautiful city. And <mask> wife <mask> to there <mask>


In [39]:
import torch
eos_mask = torch.Tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False,  True, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False]]).bool()

hidden_states = torch.LongTensor([[[i]*5 for i in range(eos_mask.size(-1))]])
# print(hidden_states)
print(hidden_states[eos_mask, :].view(hidden_states.size(0), -1, hidden_states.size(-1))[:, -1, :])
decoder_inputs = None

eos_mask = decoder_inputs.eq(tokenizer.eos_token_id)
sentence_representation = hidden_states[eos_mask, :].view(hidden_states.size(0), -1, hidden_states.size(-1))[
            :, -1, :
        ]
        
print(sentence_representation)

tensor([[53, 53, 53, 53, 53]])
tensor([[53, 53, 53, 53, 53]])


In [ ]:
import json
import gzip

file = 'dataset/wiki_NER/processed2/0.jsonl.gz'

with gzip.GzipFile(file, 'r') as fin:
    json_bytes = fin.read().splitlines()   

    for i, bytes in enumerate(json_bytes):
        json_line = json.loads(str(bytes, 'utf-8'))

        print(json_line)

        if i >= 3:
            break